In [1]:
import sys
import logging
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import json
import os

# -----------------------------
# Fix paths - Run from project root
# -----------------------------
project_root = Path.cwd()
print(f"Project root: {project_root}")
print(f"Current directory: {os.getcwd()}")

# Ensure we're in the right directory
if not (project_root / 'data').exists():
    print("WARNING: 'data' folder not found in current directory")
    print("Switching to parent directory...")
    project_root = project_root.parent
    os.chdir(project_root)
    print(f"New working directory: {os.getcwd()}")

sys.path.append(str(project_root))
sys.path.append(str(project_root / 'src'))

print(f"Project root added to sys.path: {project_root}")
print(f"Python path: {sys.path}")
# -----------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class BaselineAlgorithmTester:
    """Comprehensive testing framework for baseline algorithm."""
    
    def __init__(self, data_dir="data", num_farms=250, num_days=7):
        self.data_dir = Path(data_dir)
        self.num_farms = num_farms
        self.num_days = num_days
        self.metrics = {
            'daily_scores': [],
            'daily_deliveries': [],
            'daily_efficiency': [],
            'timestamps': []
        }
        self.algorithm = None
        
        # Create data directory if it doesn't exist
        self.data_dir.mkdir(exist_ok=True)
        print(f"Data directory: {self.data_dir.absolute()}")
    
    def setup(self):
        """Initialize data and algorithm."""
        try:
            logger.info("Loading data...")
            
            # First, check what files we have
            print("\nChecking data files...")
            required_files = [
                'parameters.json',
                'stp_registry.csv', 
                'farm_locations.csv',
                'daily_n_demand.csv',
                'weather_forecast.csv'
            ]
            
            missing_files = []
            for file in required_files:
                file_path = self.data_dir / file
                if file_path.exists():
                    print(f"✓ Found: {file}")
                else:
                    print(f"✗ Missing: {file}")
                    missing_files.append(file)
            
            # If we're missing data, create it
            if missing_files:
                logger.info(f"Creating {len(missing_files)} missing data files...")
                self.create_missing_data(missing_files)
            
            # Now load the data
            from src.utils.data_loader import DataLoader
            loader = DataLoader(data_path=str(self.data_dir))
            params, stps_df, farms_df = loader.validate_data()
            
            logger.info(f"Loaded {len(stps_df)} STPs and {len(farms_df)} farms")
            
            # Load demand and weather data
            try:
                self.demand_df = pd.read_csv(self.data_dir / 'daily_n_demand.csv')
                self.weather_df = pd.read_csv(self.data_dir / 'weather_forecast.csv')
                logger.info(f"Loaded demand data: {len(self.demand_df)} records")
                logger.info(f"Loaded weather data: {len(self.weather_df)} records")
            except Exception as e:
                logger.error(f"Failed to load data files: {e}")
                # Generate synthetic data
                from src.utils.data_simulator import DataSimulator
                simulator = DataSimulator(data_path=str(self.data_dir), num_farms=self.num_farms)
                self.demand_df = simulator.generate_daily_demand(num_days=self.num_days)
                self.weather_df = simulator.generate_weather_forecast()
            
            logger.info("Initializing algorithm...")
            from src.algorithm.greedy_baseline import GreedyBaseline
            self.algorithm = GreedyBaseline(params, stps_df, farms_df)
            self.params = params
            
            return True
        except Exception as e:
            logger.error(f"Setup failed: {e}", exc_info=True)
            return False
    
    def create_missing_data(self, missing_files):
        """Create missing data files."""
        import json
        
        # Create parameters.json if missing
        if 'parameters.json' in missing_files:
            params = {
                'transport_emission_factor': 0.9,
                'fertilizer_offset_factor': 5.0,
                'soil_sequestration_factor': 0.2,
                'excess_n_penalty': 10.0,
                'overflow_penalty_per_ton': 1000
            }
            with open(self.data_dir / 'parameters.json', 'w') as f:
                json.dump(params, f, indent=2)
            print("Created parameters.json")
        
        # Create STP data if missing
        if 'stp_registry.csv' in missing_files:
            stp_data = {
                'stp_id': ['STP-1', 'STP-2', 'STP-3', 'STP-4'],
                'latitude': [9.9312, 8.5241, 11.2588, 10.5276],
                'longitude': [76.2673, 76.9366, 75.7804, 76.2144],
                'storage_max_tons': [100, 120, 80, 90],
                'daily_output_tons': [15, 18, 12, 14]
            }
            pd.DataFrame(stp_data).to_csv(self.data_dir / 'stp_registry.csv', index=False)
            print("Created stp_registry.csv")
        
        # Create farm data if missing (use 20 farms for testing)
        if 'farm_locations.csv' in missing_files:
            farm_ids = list(range(1, 21))
            farm_data = {
                'farm_id': farm_ids,
                'latitude': [8.5 + np.random.random() * 4.5 for _ in farm_ids],
                'longitude': [74.5 + np.random.random() * 3.0 for _ in farm_ids],
                'weather_zone': np.random.choice(['North', 'Central', 'South'], size=len(farm_ids))
            }
            pd.DataFrame(farm_data).to_csv(self.data_dir / 'farm_locations.csv', index=False)
            print("Created farm_locations.csv (20 farms)")
    
    def run_simulation(self):
        """Run the simulation for all days."""
        if not self.algorithm:
            logger.error("Algorithm not initialized. Call setup() first.")
            return False
        
        logger.info("\n" + "="*60)
        logger.info(f"RUNNING {self.num_days}-DAY SIMULATION")
        logger.info("="*60 + "\n")
        
        # Get unique dates
        dates = sorted(self.demand_df['date'].unique())[:self.num_days]
        
        total_score = 0
        total_deliveries = 0
        
        for i, date in enumerate(dates, 1):
            try:
                print(f"\n📅 Day {i}: {date}")
                daily_score = self.algorithm.run_day(date, self.demand_df, self.weather_df)
                
                # Count deliveries for this day
                day_deliveries = [d for d in self.algorithm.deliveries if d.get('date') == date]
                num_deliveries = len(day_deliveries)
                
                # Calculate efficiency (deliveries per unit score)
                efficiency = num_deliveries / (abs(daily_score) + 1e-6) if daily_score != 0 else 0
                
                total_score += daily_score
                total_deliveries += num_deliveries
                
                # Store metrics
                self.metrics['daily_scores'].append(daily_score)
                self.metrics['daily_deliveries'].append(num_deliveries)
                self.metrics['daily_efficiency'].append(efficiency)
                self.metrics['timestamps'].append(date)
                
                logger.info(f"  Score: {daily_score:>8.2f} | "
                           f"Deliveries: {num_deliveries:>3} | "
                           f"Efficiency: {efficiency:>6.3f}")
                
            except Exception as e:
                logger.error(f"Error on day {i} ({date}): {e}", exc_info=True)
                return False
        
        logger.info("\n" + "="*60)
        logger.info("SIMULATION COMPLETE")
        logger.info("="*60)
        logger.info(f"Total CO2eq Score: {total_score:.2f}")
        logger.info(f"Total Deliveries: {total_deliveries}")
        logger.info(f"Average per day: {total_deliveries/self.num_days:.1f}")
        logger.info("="*60 + "\n")
        
        self.total_score = total_score
        self.total_deliveries = total_deliveries
        
        return True
    
    def get_solution_csv(self, save=True):
        """Retrieve and display solution data."""
        try:
            solution_df = self.algorithm.get_solution_csv()
            
            if solution_df.empty:
                logger.warning("No deliveries recorded in solution.csv")
                return None
            
            logger.info(f"Solution CSV shape: {solution_df.shape}")
            logger.info(f"\nFirst 5 rows:")
            print(solution_df.head().to_string())
            
            if save:
                # Create output directory
                output_dir = Path('output')
                output_dir.mkdir(exist_ok=True)
                
                # Save solution
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                output_path = output_dir / f'solution_{timestamp}.csv'
                solution_df.to_csv(output_path, index=False)
                logger.info(f"\n💾 Saved to: {output_path}")
                
                # Also save as the main solution.csv
                main_path = output_dir / 'solution.csv'
                solution_df.to_csv(main_path, index=False)
                logger.info(f"💾 Also saved as: {main_path}")
            
            return solution_df
            
        except Exception as e:
            logger.error(f"Failed to get solution CSV: {e}")
            return None
    
    def generate_summary_metrics(self):
        """Generate summary_metrics.json in required format."""
        try:
            # Calculate metrics
            total_tons = sum(d.get('tons_delivered', 0) for d in self.algorithm.deliveries)
            
            # Carbon breakdown (simplified)
            carbon_breakdown = {
                'fertilizer_offset': 0,
                'soil_sequestration': 0,
                'transport_cost': 0,
                'excess_penalty': 0,
                'overflow_penalty': 0
            }
            
            for delivery in self.algorithm.deliveries:
                if 'carbon_impact' in delivery:
                    impact = delivery['carbon_impact']
                    for key in carbon_breakdown:
                        if key in impact:
                            carbon_breakdown[key] += impact[key]
            
            summary = {
                "total_carbon_credits_kg": round(self.total_score, 2),
                "total_deliveries": self.total_deliveries,
                "total_tons_delivered": round(total_tons, 2),
                "average_tons_per_delivery": round(total_tons / self.total_deliveries, 3) if self.total_deliveries > 0 else 0,
                "simulation_days": self.num_days,
                "average_daily_carbon": round(self.total_score / self.num_days, 2),
                "carbon_breakdown": {k: round(v, 2) for k, v in carbon_breakdown.items()}
            }
            
            # Save to JSON
            output_dir = Path('output')
            output_dir.mkdir(exist_ok=True)
            
            summary_path = output_dir / 'summary_metrics.json'
            with open(summary_path, 'w') as f:
                json.dump(summary, f, indent=2)
            
            logger.info(f"📊 Summary metrics saved to: {summary_path}")
            logger.info("\nSummary:")
            for key, value in summary.items():
                if key != 'carbon_breakdown':
                    logger.info(f"  {key}: {value}")
            
            return summary
            
        except Exception as e:
            logger.error(f"Failed to generate summary metrics: {e}")
            return None
    
    def validate_results(self):
        """Validate simulation results for consistency."""
        logger.info("\nValidating results...")
        
        issues = []
        
        # Check solution.csv format
        solution_df = self.algorithm.get_solution_csv()
        if not solution_df.empty:
            required_columns = ['date', 'stp_id', 'farm_id', 'tons_delivered']
            missing_columns = [col for col in required_columns if col not in solution_df.columns]
            
            if missing_columns:
                issues.append(f"Missing columns in solution: {missing_columns}")
            
            # Check for negative tons
            if (solution_df['tons_delivered'] < 0).any():
                issues.append("Negative tons_delivered values found")
            
            # Check for reasonable values
            max_tons = solution_df['tons_delivered'].max()
            if max_tons > 10:  # Truck capacity
                issues.append(f"Delivery exceeds truck capacity: {max_tons} tons")
        
        # Check carbon score sign (should be positive)
        if self.total_score < 0:
            issues.append(f"Total carbon score is negative: {self.total_score}")
        
        if issues:
            logger.warning(f"Found {len(issues)} issues:")
            for issue in issues:
                logger.warning(f"  - {issue}")
            return False
        else:
            logger.info("✓ All validations passed")
            return True


def main():
    """Main execution function."""
    print("\n" + "="*70)
    print("STEP 8: COMPREHENSIVE ALGORITHM TEST")
    print("="*70)
    
    try:
        # Initialize tester
        tester = BaselineAlgorithmTester(data_dir="data", num_farms=20, num_days=3)
        
        print("\n1️⃣ SETUP")
        print("-"*40)
        if not tester.setup():
            logger.error("Setup failed. Exiting.")
            return 1
        
        print("\n2️⃣ SIMULATION")
        print("-"*40)
        if not tester.run_simulation():
            logger.error("Simulation failed. Exiting.")
            return 1
        
        print("\n3️⃣ OUTPUT GENERATION")
        print("-"*40)
        solution_df = tester.get_solution_csv(save=True)
        
        print("\n4️⃣ METRICS GENERATION")
        print("-"*40)
        tester.generate_summary_metrics()
        
        print("\n5️⃣ VALIDATION")
        print("-"*40)
        tester.validate_results()
        
        print("\n" + "="*70)
        print("🎉 TEST COMPLETED SUCCESSFULLY!")
        print("="*70)
        
        print("\n📁 GENERATED FILES:")
        output_dir = Path('output')
        if output_dir.exists():
            for file in output_dir.iterdir():
                print(f"  - {file.name} ({file.stat().st_size} bytes)")
        
        print("\n✅ READY FOR NEXT STEPS:")
        print("   1. Run 7-day full simulation")
        print("   2. Connect dashboard to real data")
        print("   3. Implement Triple-Lens algorithm")
        
        return 0
        
    except KeyboardInterrupt:
        logger.info("Test interrupted by user")
        return 1
    except Exception as e:
        logger.error(f"Unexpected error: {e}", exc_info=True)
        return 1


if __name__ == "__main__":
    exit_code = main()
    sys.exit(exit_code)

2026-01-14 22:58:31,205 - INFO - Loading data...
2026-01-14 22:58:31,214 - INFO - Loaded config.json
2026-01-14 22:58:31,214 - INFO - Loaded config.json
2026-01-14 22:58:31,225 - INFO - Loaded stp_registry.csv (4 records)
2026-01-14 22:58:31,225 - INFO - Loaded stp_registry.csv (4 records)
2026-01-14 22:58:31,230 - INFO - Loaded farm_locations.csv (20 records)
2026-01-14 22:58:31,230 - INFO - Loaded farm_locations.csv (20 records)
2026-01-14 22:58:31,241 - INFO - Loaded 4 STPs and 20 farms
2026-01-14 22:58:31,302 - INFO - Loaded demand data: 91250 records
2026-01-14 22:58:31,303 - INFO - Loaded weather data: 1825 records
2026-01-14 22:58:31,304 - INFO - Initializing algorithm...


Project root: c:\Users\Prasanna rajan R\Documents\GitHub\kerala-biosolids-hackathon\notebooks
Current directory: c:\Users\Prasanna rajan R\Documents\GitHub\kerala-biosolids-hackathon\notebooks
Switching to parent directory...
New working directory: c:\Users\Prasanna rajan R\Documents\GitHub\kerala-biosolids-hackathon
Project root added to sys.path: c:\Users\Prasanna rajan R\Documents\GitHub\kerala-biosolids-hackathon
Python path: ['C:\\Users\\Prasanna rajan R\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\Prasanna rajan R\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\Prasanna rajan R\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\Prasanna rajan R\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\Prasanna rajan R\\Documents\\GitHub\\kerala-biosolids-hackathon\\.venv', '', 'c:\\Users\\Prasanna rajan R\\Documents\\GitHub\\kerala-biosolids-hackathon\\.venv\\Lib\\site-packages', 'c:\\Users\\Prasanna rajan R\\Documents\\

2026-01-14 22:58:31,505 - INFO - 
2026-01-14 22:58:31,506 - INFO - RUNNING 3-DAY SIMULATION
2026-01-14 22:58:31,507 - INFO - ============================================================

2026-01-14 22:58:31,604 - INFO -   Score: 19286.02 | Deliveries:  16 | Efficiency:  0.001
2026-01-14 22:58:31,644 - INFO -   Score: 38760.15 | Deliveries:  17 | Efficiency:  0.000
2026-01-14 22:58:31,687 - INFO -   Score: 58307.85 | Deliveries:  16 | Efficiency:  0.000



2️⃣ SIMULATION
----------------------------------------

📅 Day 1: 2025-01-01

📅 Day 2: 2025-01-02

📅 Day 3: 2025-01-03


2026-01-14 22:58:31,690 - INFO - 
2026-01-14 22:58:31,690 - INFO - SIMULATION COMPLETE
2026-01-14 22:58:31,691 - INFO - ============================================================
2026-01-14 22:58:31,692 - INFO - Total CO2eq Score: 116354.01
2026-01-14 22:58:31,693 - INFO - Total Deliveries: 49
2026-01-14 22:58:31,696 - INFO - Average per day: 16.3
2026-01-14 22:58:31,697 - INFO - ============================================================

2026-01-14 22:58:31,703 - INFO - Solution CSV shape: (49, 4)
2026-01-14 22:58:31,704 - INFO - 
First 5 rows:
2026-01-14 22:58:31,713 - INFO - 
💾 Saved to: output\solution_20260114_225831.csv
2026-01-14 22:58:31,719 - INFO - 💾 Also saved as: output\solution.csv
2026-01-14 22:58:31,721 - ERROR - Failed to generate summary metrics: argument of type 'numpy.float64' is not iterable
2026-01-14 22:58:31,722 - INFO - 
Validating results...
2026-01-14 22:58:31,725 - INFO - ✓ All validations passed



3️⃣ OUTPUT GENERATION
----------------------------------------
         date stp_id  farm_id  tons_delivered
0  2025-01-01  STP-1       15           5.049
1  2025-01-01  STP-1        9           6.517
2  2025-01-01  STP-1       12           3.434
3  2025-01-01  STP-2       19           5.898
4  2025-01-01  STP-2        2           4.096

4️⃣ METRICS GENERATION
----------------------------------------

5️⃣ VALIDATION
----------------------------------------

🎉 TEST COMPLETED SUCCESSFULLY!

📁 GENERATED FILES:
  - solution.csv (1313 bytes)
  - solution_20260114_225831.csv (1313 bytes)
  - solution_step8_test.csv (356 bytes)

✅ READY FOR NEXT STEPS:
   1. Run 7-day full simulation
   2. Connect dashboard to real data
   3. Implement Triple-Lens algorithm


SystemExit: 0

c:\Users\Prasanna rajan R\Documents\GitHub\kerala-biosolids-hackathon\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
